In [1]:
# *** OpenGamma_verbose.ipynb ***
# 2020-09-21
# Essa é a versão verbose, ou seja, com comentários, gráficos etc.
# Agora vou congelar por enquanto e fazer em lotes
# 2020-12-05 Descongelando!!

# Em setembro/2020, trabalhei bastante com esta versão, inluindo o cálculo correto
# da linha base usando percentis e extrapolando para a mediana da linha base
# usando a função erf
# Usei o espectro CTP2203-3.Chn, que tem altos picos de I-131 distorcidos e grandes platôs Compton.
# E fiz a recostrução do espectro com a análise dos picos.
# Obtive bons resultados, mas há como melhorar

# 2020-12-05
# Agora em dezembro/2020 vou usar para espectros "comuns" de efluente


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do picoh
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np
import numpy.ma as ma  # masked array
import scipy.stats as sta # statistics
from scipy.special import (erf, erfinv)

from numpy import linalg as LA

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from gamma_spectrum_class import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.models import ColumnDataSource, Whisker, Label, LabelSet, MultiLine
from bokeh.palettes import Paired6
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot, column, row

from bokeh import palettes

# 2020-08-22 Para rodar curve_fit
from gauss_funcs import func_example, func_example_fixed_b
# 2020-09-03 Desativei por enquanto
# import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# 2020-08-30
from scipy.interpolate import splrep, splev

from scipy.ndimage import label, generate_binary_structure, find_objects, binary_dilation  # 2020-09-03
from numpy.polynomial import Polynomial as P # 2020-09-06 Esta é a nova classe recomendada 

# 2020-09-11
from opengamma_searchfuncs import selectOverlappedPks

# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [3]:

# hei = 0.01
# haPercWin = 9 # 2020-12-05 Meia janela para o cálculo do percentil para a linha base:
# 9 para 4k canais, 18 para 8k canais.
# yrange = [10**1, 10**5]
# fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'
# yrange = [10**1, 10**5]
# fn2 = '../Genie_Transfer/Si/SI2019/SI04019.Chn'
# fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

# Para espectros CTP 4k
kSigma = 4.0 # Com 3.0 ele pega os grandes degraus Compton, com 4.0 só o da esquerda (retroespalhamento)
aSta = 0.004
bSta = 3.0
kExpand = 1 # Parâmetro para alargar região (inteiro positivo)
# 2020-12-05 Ainda não aprendi a mexer com o smoo... Chutando bem baixo para 4k, altíssima intens
smoo = 0.1 # Tem que ser >= 0.0
yrange = [10**1, 10**6]
fn2 = '../Ipen/Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'

# Para espectros Almera 8k
# kSigma = 3.0 # Maior, menos sensível
# aSta = 0.003 # Janela para estatísticas
# bSta = 10.0
# kExpand = 1 # Parâmetro para alargar região (inteiro positivo)
# smoo = 15
# yrange = [10**1, 10**5]

# Alguns espectros ALMERA com relatórios do Genie2k em .REP:
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp3(QC)_F100_16_Cont-A.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp2_spk_water_16_250k.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp4_f100_8k.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp4_f100_63k.IEC'
# fn2 = '../gamma/spectra/ALMERA-2016/alm_samp5_F100_16_Cont-A.IEC'

# kSigma = 2.0
# fn2 = '../Pesquisa/2020_Mariana_PR/2020_Mariana_PR_Espectros/espectros-det-projeto/agua-bruta.IEC'

# yrange = [10**1, 10**6]
# fn2 = './I125_01.001.3.IEC'

# 2020-12-05 Espectros do tq de retenção do CEN: 54 e 55 de 2020.
# yrange = [10**1, 10**3]
# fn2 = '../Genie_Transfer/Si/SI2020/SI05420.Chn'

# yrange = [10**1, 10**5]
# fn2 = '../Genie_Transfer/Si/SI2019/SI04019.Chn'

# sigma 5 funciona para este
# yrange = [10**1, 10**6]
# fn2 = './I125_01.001.3.IEC'

# 13 de julho de 2020
# PAREI AQUI O termo de 1o grau oscila selvagemente em baixas contagens
# ponderá-lo pelo coeffs0
# Para o espectro do I-123, usar kSigma = 5.0


In [4]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.nCh)
    
counts = spec2.specIO.spCounts
print(len(counts))

4096
4096


In [5]:
# 2020-12-15
# A nomenclatura dos arrays está algo incoerente.
if (spec2.chancalib.size >= 3) & (spec2.enChcalib.size == spec2.chancalib.size):
    spec2.coeffs_ChEn = P.fit(spec2.chancalib, spec2.enChcalib, deg=2)
if (spec2.enFwcalib.size >= 3) & (spec2.fwhmcalib.size == spec2.enFwcalib.size):
    spec2.coeffs_EnFw = P.fit(spec2.enFwcalib, spec2.fwhmcalib, deg=2)

In [6]:
print ('\nChannel vs Energy')
print (spec2.enChcalib)
print (spec2.chancalib)
print (spec2.coeffs_ChEn)

print ('\nEnergy vs FWHM')
print (spec2.enFwcalib)
print (spec2.fwhmcalib)
print (spec2.coeffs_EnFw)

print ('\nEnergy vs Efficiency')
print (spec2.enEfcalib)
print (spec2.efficalib)
print (spec2.coeffs_EnEf)


Channel vs Energy
[]
[]
[0.0, 0.8981618285179138, -7.436953544616699]

Energy vs FWHM
[]
[]
[0.0, 0.0008241464965976775, 1.392101764678955]

Energy vs Efficiency
[]
[]
[]


In [7]:
spec2.pEn(2342)

2096.0580488443375

In [8]:
# Atenção: isso parece estar duplicado? 
print(spec2.pFw)

poly([1.39210176e+00 8.24146497e-04 0.00000000e+00])


In [9]:
xs = spec2.xs
y0s = spec2.y0sCorr
lnCounts = spec2.lnCounts
nCh = xs.size
nCh

4096

In [10]:
spec2.countDatetime

datetime.datetime(2018, 4, 10, 10, 58, 11)

In [11]:
spec2.spLVTime

5000.0

In [12]:
test_chs = np.linspace( 0, nCh-1, 10 )
[spec2.hStaW(ch,a=aSta,b=bSta) for ch in test_chs]

[3, 5, 7, 8, 10, 12, 14, 16, 18, 19]

In [13]:
# Calcular espectro com ajuste parabólico na mesma janela e calculado no centro da janela
parabSpec = np.zeros(nCh)
hWIni = spec2.hStaW(0,a=aSta,b=bSta)
hWFin = spec2.hStaW(nCh-1,a=aSta,b=bSta)
for i in range(hWIni,nCh-hWFin):
    hW = spec2.hStaW(i,a=aSta,b=bSta)
    chanWin = np.array(range(i-hW,i+hW+1))
    lnctsWindow = lnCounts[chanWin]
    # Primeiro, acha usando parábola na janela e calcula no canal central
    parabWinFit = P.fit(chanWin,lnctsWindow,deg=2)
    parabSpec[i]=parabWinFit(i)
tckDirectSpl = splrep(xs, parabSpec, k=3, s=smoo)
smooParabSpec = np.exp(splev(xs, tckDirectSpl, der=0))

In [14]:
del parabSpec

In [15]:
# 2020-12-16 
# Agora, smooParabSpec
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok1b = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok1b.title.text = 'Gráfico 1b: smooParabSpec (blu)'
bok1b.circle(spec2.xs, spec2.y0s, color="red",size=2)
bok1b.line(spec2.xs, smooParabSpec, color="blue")
# set up layout
p = gridplot([bok1b], ncols=1)
show(p)

In [16]:
isRegionSigma = np.abs(y0s-smooParabSpec) / (np.sqrt(y0s+5))
isReg = isRegionSigma > kSigma
isReg

array([ True,  True,  True, ..., False, False, False])

In [17]:
# 2020-12-06
# Expandindo canais marcados para chExpand = kExpand * hStaW(ch) canais antes e depois
# Deu certo: depois, eliminar chExpand fixo.

isRegExpand = np.zeros(nCh)
chExpandIni = kExpand*spec2.hStaW(0,a=aSta,b=bSta)
chExpandFin = kExpand*spec2.hStaW(nCh-1,a=aSta,b=bSta)

for i in range(chExpandIni, nCh-chExpandFin+1):
    chExpandMov = kExpand*spec2.hStaW(i,a=aSta,b=bSta)
    isRegExpand[i] = np.any(isReg[i-chExpandMov:i+chExpandMov+1])
isRegExpand

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
# Para plotar
isRegPlot = isRegExpand * 1e3 + 1e2
isRegPlot

array([100., 100., 100., ..., 100., 100., 100.])

In [19]:
# 2020-09-20 
# Gráfico 2: Canais marcados
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok2 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok2.title.text = 'Gráfico 2: Canais marcados'
bok2.line(xs, isRegPlot, color='black')
bok2.line(xs, y0s, color='lightblue')
# set up layout
p = gridplot([bok2], ncols=1)
show(p)

In [20]:
# Com base nos canais marcados, definir extremos das regiões

In [21]:
comuta = np.zeros(nCh)
for i in range(1,nCh):
    comuta[i] = isRegExpand[i] - isRegExpand[i-1]
comuta

array([0., 0., 0., ..., 0., 0., 0.])

In [22]:
# inis = np.insert(np.nonzero(comuta>0), 0, 0)
# np.nonzero gera uma tupla, não sei por quê.
inis = np.nonzero(comuta>0)[0]
inis

array([  3,  91, 170, 200, 309, 357, 401, 455, 561, 706, 804])

In [23]:
# fins = np.append(np.nonzero(comuta<0), n)
fins = np.nonzero(comuta<0)[0]
fins

array([  8, 108, 183, 214, 336, 381, 429, 466, 579, 736, 830])

In [24]:
# Ajusta comprimento dos arrays. Têm de ser iguais.
minSize = np.minimum(inis.size, fins.size)
inis = inis[:minSize] 
fins = fins[:minSize] 

In [25]:
mix = np.concatenate(np.array([[inis], [fins]])).T
mix

array([[  3,   8],
       [ 91, 108],
       [170, 183],
       [200, 214],
       [309, 336],
       [357, 381],
       [401, 429],
       [455, 466],
       [561, 579],
       [706, 736],
       [804, 830]])

In [26]:
mix.shape

(11, 2)

In [27]:
# Agora, criar nova baseline onde
# Fora das regiões marcadas, seja smooParabSpec
# Nas regiões marcadas, seja a stepLine dos extremos

In [28]:
assembledBaseLine = np.array(smooParabSpec)
for regLim in mix:
    assembledBaseLine[regLim[0]:regLim[1]+1] = spec2.stepBaseLine (
        regLim[0], regLim[1], smooParabSpec[regLim[0]], smooParabSpec[regLim[1]], y0s)
assembledBaseLine

array([1.00029724, 0.99755678, 1.01162162, ..., 1.00030969, 0.99992796,
       0.99931596])

In [29]:
assembledBaseLine.shape

(4096,)

In [30]:
# 2020-12-16
# Substituindo smooParabSpec por assembledBaseLine
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok3 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok3.title.text = 'Gráfico 3: assembledBaseLine'
bok3.circle(xs, y0s, color='red',size=2)
bok3.line(xs, assembledBaseLine, color='green')
# set up layout
p = gridplot([bok3], ncols=1)
show(p)

In [31]:
lnNetCounts = np.zeros(nCh)
wLnNetCounts = np.zeros(nCh)
uncys = np.sqrt(y0s)
contin = assembledBaseLine
yNets = y0s - contin

for i in range(nCh):
    if yNets[i] < 2:
        lnNetCounts[i] = 0
        wLnNetCounts[i] = 0
    else:
        lnNetCounts[i] = np.log(yNets[i])
        wLnNetCounts[i] = np.sqrt(y0s[i])

In [32]:
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok4 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok4.title.text = 'Gráfico 4: yNets'
bok4.line(xs, y0s, color="red")
bok4.line(xs, yNets, color="blue")
# set up layout
p = gridplot([bok4], ncols=1)
show(p)

In [33]:
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok4b = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    # y_axis_type="log",
    tools=tools
)
bok4b.title.text = 'Gráfico 4b: yNets linear'
bok4b.line(xs, y0s, color="red")
bok4b.line(xs, yNets, color="blue")
# set up layout
p = gridplot([bok4b], ncols=1)
show(p)

In [34]:
mix

array([[  3,   8],
       [ 91, 108],
       [170, 183],
       [200, 214],
       [309, 336],
       [357, 381],
       [401, 429],
       [455, 466],
       [561, 579],
       [706, 736],
       [804, 830]])

In [35]:
mix.shape

(11, 2)

In [36]:
# 2020-12-16
# Usando as regiões mix, ir direto ao ajuste parabólico
# Quebrar yNets usando mix
# Testar se é uma gaussiana simples ou multipleto

In [37]:
assembledBaseLine.shape

(4096,)

In [38]:
len(mix)

11

In [39]:
# 2020-12-16  Agora vou usar split para dividir o array. Vejamos:

In [40]:
regLim = mix[4]
teste = yNets[regLim[0]:regLim[1]]
teste

array([-5.28378236e+01, -7.58279830e+01,  8.20255665e+01,  7.10277505e+02,
        1.07836816e+03,  5.21494636e+02, -1.57180838e+02,  6.97399773e+00,
       -8.54513254e+01,  3.75560733e+00,  2.04152541e+02,  8.73079639e+02,
        8.61971104e+02,  1.29073692e+03,  2.05772549e+03,  1.33639696e+04,
        8.41707601e+04,  1.35300333e+05,  5.69067216e+04,  6.59352098e+03,
        2.98524623e+02, -1.28978147e+01, -7.83499356e+01, -1.77074800e+02,
       -1.11735374e+01,  1.50165953e+02, -4.70716500e+01])

In [41]:
maxCountInReg = np.max(teste)

In [42]:
# 2020-12-18 AQUI: 1e-3 picota demais. Usar:
cutFactor = 1e-5

In [43]:
isToSplit = (teste / maxCountInReg) < cutFactor
isToSplit

array([ True,  True, False, False, False, False,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True, False,  True])

In [44]:
isToSplit.size

27

In [45]:
# 2020-12-17 Interessante, não conhecia esse operador lógico: ^
comuta = np.zeros(isToSplit.size)
for i in range(1,isToSplit.size):
    comuta[i] = isToSplit[i] ^ isToSplit[i-1]
comuta

array([0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 1.])

In [46]:
comuta.size

27

In [47]:
splitting, = comuta.nonzero() # Precisa da vírgula pois é tupla
splitting

array([ 2,  6,  7,  8,  9, 21, 25, 26])

In [48]:
splitted = np.split(teste,splitting)

In [49]:
# agora uso isToSplit para ver quais interessam
isToSplit

array([ True,  True, False, False, False, False,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True, False,  True])

In [50]:
# Lembrar que len(splitted) == splitting.size
usefulSubRegions = []
if not isToSplit[0]:
    usefulSubRegions.append(splitted[0])
for i in range(splitting.size):
    print (isToSplit[splitting[i]])
    if not isToSplit[splitting[i]]:
        usefulSubRegions.append(splitted[i+1])    

False
True
False
True
False
True
False
True


In [51]:
# Ok. Agora, montando tudo: e acrescentando os canais xs correpondentes, e também:
uncys # Defini lá atrás, porque se baseia direto em y0s

array([154.28869045, 154.28869045, 154.28869045, ...,   0.        ,
         0.        ,   0.        ])

In [52]:
xsUsefulSubRegions = []
usefulSubRegions = []
wsUsefulSubRegions = []
for regLim in mix:
    # print('===============')
    xsNetReg = xs[regLim[0]:regLim[1]]
    netRegion = yNets[regLim[0]:regLim[1]]
    #  wsNetReg = uncys[regLim[0]:regLim[1]]  # Nao bom com uncys. Tentar com:
    netRegion[netRegion < 0.0] = 0.0
    wsNetReg = netRegion
    maxCountInReg = np.max(netRegion)
    isToSplit = (netRegion / maxCountInReg) < cutFactor
    comuta = np.zeros(isToSplit.size)
    for i in range(1,isToSplit.size):
        comuta[i] = isToSplit[i] ^ isToSplit[i-1]
    splitting, = comuta.nonzero() # Precisa da vírgula pois é tupla
    xsSplitted = np.split(xsNetReg,splitting)
    splitted = np.split(netRegion,splitting)
    wsSplitted = np.split(wsNetReg,splitting)
    # print(splitted)
    # print('-----')
    # for i in range(0,len(splitted),2):
    #     print(splitted[i])
    # for j in range(1,len(splitted),2):
    #     print(splitted[j])
    if not isToSplit[0]:
        xsUsefulSubRegions.append(xsSplitted[0])
        usefulSubRegions.append(splitted[0])
        wsUsefulSubRegions.append(wsSplitted[0])
    for i in range(splitting.size):
        if not isToSplit[splitting[i]]:
            xsUsefulSubRegions.append(xsSplitted[i+1])
            usefulSubRegions.append(splitted[i+1])
            wsUsefulSubRegions.append(wsSplitted[i+1])
# print('xsUsefulSubRegions:')
# print(xsUsefulSubRegions)
# print('usefulSubRegions:')
# usefulSubRegions

In [53]:
len(usefulSubRegions)

27

In [54]:
len(xsUsefulSubRegions)

27

In [55]:
len(wsUsefulSubRegions)

27

In [56]:
netRegion

array([0.00000000e+00, 4.73838399e+00, 4.44051237e+01, 4.42087206e+01,
       1.03008808e+02, 1.57012427e+02, 1.92202031e+02, 4.21510946e+02,
       2.38461995e+03, 9.85561041e+03, 1.49817862e+04, 8.22882709e+03,
       1.70298468e+03, 2.92139737e+02, 2.37322322e+02, 2.62308395e+02,
       3.51378688e+02, 2.37757787e+02, 6.87333321e+01, 1.91123210e+01,
       1.03158520e+01, 1.34878007e+01, 8.67027686e+00, 0.00000000e+00,
       9.51009379e-01, 0.00000000e+00])

In [57]:
netRegion[netRegion<0.0] = 0.0
netRegion

array([0.00000000e+00, 4.73838399e+00, 4.44051237e+01, 4.42087206e+01,
       1.03008808e+02, 1.57012427e+02, 1.92202031e+02, 4.21510946e+02,
       2.38461995e+03, 9.85561041e+03, 1.49817862e+04, 8.22882709e+03,
       1.70298468e+03, 2.92139737e+02, 2.37322322e+02, 2.62308395e+02,
       3.51378688e+02, 2.37757787e+02, 6.87333321e+01, 1.91123210e+01,
       1.03158520e+01, 1.34878007e+01, 8.67027686e+00, 0.00000000e+00,
       9.51009379e-01, 0.00000000e+00])

In [58]:
newregsMarks = [np.ones_like(x)*1.0e4 for x in xsUsefulSubRegions]

In [59]:
usefulSubRegions[0:13]

[array([1056.79673976,  883.55313808,  710.30953639,  537.06593471,
         363.82233303]),
 array([5.77234058e+02, 7.87291748e+02, 8.27677337e+02, 8.02339073e+02,
        1.55787481e+03, 9.71812263e+03, 4.99688709e+04, 5.62223504e+04,
        1.34096946e+04, 3.11201741e+01]),
 array([383.55925796]),
 array([35.07484219]),
 array([21.37158879]),
 array([ 138.40050748, 1214.35754157, 2336.84170323, 3200.3047988 ,
        3102.15624063, 2210.66079463, 1224.14361846,  650.54485355]),
 array([622.81194835, 206.30162231,  31.18730476, 226.78605849]),
 array([1246.00714912, 6365.02613451, 8836.69295704, 2521.53040745,
         176.02203157]),
 array([149.13602573,  73.28931934]),
 array([  82.02556648,  710.27750533, 1078.36816457,  521.4946363 ]),
 array([6.97399773]),
 array([3.75560733e+00, 2.04152541e+02, 8.73079639e+02, 8.61971104e+02,
        1.29073692e+03, 2.05772549e+03, 1.33639696e+04, 8.41707601e+04,
        1.35300333e+05, 5.69067216e+04, 6.59352098e+03, 2.98524623e+02]),
 array

In [60]:
wsUsefulSubRegions[0:13]

[array([1056.79673976,  883.55313808,  710.30953639,  537.06593471,
         363.82233303]),
 array([5.77234058e+02, 7.87291748e+02, 8.27677337e+02, 8.02339073e+02,
        1.55787481e+03, 9.71812263e+03, 4.99688709e+04, 5.62223504e+04,
        1.34096946e+04, 3.11201741e+01]),
 array([383.55925796]),
 array([35.07484219]),
 array([21.37158879]),
 array([ 138.40050748, 1214.35754157, 2336.84170323, 3200.3047988 ,
        3102.15624063, 2210.66079463, 1224.14361846,  650.54485355]),
 array([622.81194835, 206.30162231,  31.18730476, 226.78605849]),
 array([1246.00714912, 6365.02613451, 8836.69295704, 2521.53040745,
         176.02203157]),
 array([149.13602573,  73.28931934]),
 array([  82.02556648,  710.27750533, 1078.36816457,  521.4946363 ]),
 array([6.97399773]),
 array([3.75560733e+00, 2.04152541e+02, 8.73079639e+02, 8.61971104e+02,
        1.29073692e+03, 2.05772549e+03, 1.33639696e+04, 8.41707601e+04,
        1.35300333e+05, 5.69067216e+04, 6.59352098e+03, 2.98524623e+02]),
 array

In [61]:
# 2020-09-20 
# Gráfico 2.extra: Novos canais marcados
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok2e = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok2e.title.text = 'Gráfico 2: Canais marcados'
bok2e.line(xs, y0s, color='lightblue')
bok2e.multi_line(xsUsefulSubRegions, newregsMarks, color='red')
# set up layout
p = gridplot([bok2e], ncols=1)
show(p)

In [62]:
xspolys = []
polys = []
for ixs, xsreg in enumerate(xsUsefulSubRegions):
    ysreg=usefulSubRegions[ixs]
    wsreg=wsUsefulSubRegions[ixs]
    if xsreg.size > 2:
        xspolys.append(xsreg)
        polys.append(P.fit(xsreg,np.log(ysreg),w=wsreg,deg=2))

In [63]:
len(polys)

14

In [64]:
xplots = []
y1plots = []
y2plots = []
for ixs, xspoly in enumerate(xspolys):
    xplots.append(xspoly)
    blplot = assembledBaseLine[ np.asarray(xspoly,dtype=int) ]
    y1plots.append(blplot)
    y2plots.append(np.exp(polys[ixs](xspoly)) + blplot)

In [65]:
# 2020-12-17
# Substituindo smooParabSpec por assembledBaseLine
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok3a = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type='log',
    tools=tools
)
bok3a.title.text = 'Gráfico 3a: assembledBaseLine + novo ajuste'
bok3a.circle(xs, y0s, color='red',size=2)
# bok3a.line(xs, assembledBaseLine, color='green')
bok3a.multi_line(xplots, y1plots, color='green')
bok3a.multi_line(xplots, y2plots, color='blue')
# set up layout
p = gridplot([bok3a], ncols=1)
show(p)

In [66]:
# 2020-12-18
# PAREI AQUI: Agora, partir direto para o cálculo dos picos a partir de polys
# Está tudo na list polys

In [67]:
polys

[Polynomial([ 6.56253906, -0.52218191, -0.12447678], domain=[3., 7.], window=[-1.,  1.]),
 Polynomial([ 9.26585481,  7.38893244, -8.09285163], domain=[ 92., 101.], window=[-1.,  1.]),
 Polynomial([ 8.08437147e+00, -6.89220621e-03, -1.84683799e+00], domain=[175., 182.], window=[-1.,  1.]),
 Polynomial([ 5.02450218, -0.51052478,  0.90048343], domain=[200., 203.], window=[-1.,  1.]),
 Polynomial([ 9.07723759, -0.90539189, -3.01280469], domain=[205., 209.], window=[-1.,  1.]),
 Polynomial([ 6.9179889 ,  0.66924744, -1.34287843], domain=[311., 314.], window=[-1.,  1.]),
 Polynomial([  8.56218017,  15.18697611, -17.8063094 ], domain=[318., 329.], window=[-1.,  1.]),
 Polynomial([ 6.50662401, -3.42624859, -3.82712566], domain=[362., 367.], window=[-1.,  1.]),
 Polynomial([ 8.43695527,  1.30591777, -3.74042525], domain=[369., 374.], window=[-1.,  1.]),
 Polynomial([ 13.55082317,  13.01152201, -74.54303396], domain=[402., 426.], window=[-1.,  1.]),
 Polynomial([ 6.53496905, -1.09002757, -5.6208

In [68]:
# 2020-12-21
polys_conv = []
for pol in polys:
    polconv = pol.convert().coef
    if polconv[2] < 0:
        polys_conv.append(polconv)
polys_conv_ar = np.array(polys_conv)
polys_conv_ar

array([[ 7.09001397e+00,  5.01009913e-02, -3.11191946e-02],
       [-3.87079842e+03,  7.87738548e+01, -3.99646994e-01],
       [-4.79518974e+03,  5.38201665e+01, -1.50762285e-01],
       [-3.21711317e+04,  3.11372589e+02, -7.53201172e-01],
       [-5.84171627e+04,  3.73467951e+02, -5.96834858e-01],
       [-6.24870335e+04,  3.83610266e+02, -5.88638327e-01],
       [-8.08496058e+04,  4.45025438e+02, -6.12340106e-01],
       [-8.27815433e+04,  4.45184121e+02, -5.98468041e-01],
       [-8.91601929e+04,  4.29706739e+02, -5.17659958e-01],
       [-5.87437156e+04,  2.55400754e+02, -2.77571099e-01],
       [-1.84336630e+05,  6.47697170e+02, -5.68921857e-01],
       [-9.41290415e+04,  2.62056482e+02, -1.82370920e-01],
       [-1.57819623e+05,  3.87790149e+02, -2.38202568e-01]])

In [69]:
a0 = polys_conv_ar[:,0]
a1 = polys_conv_ar[:,1]
a2 = polys_conv_ar[:,2]
a2

array([-0.03111919, -0.39964699, -0.15076228, -0.75320117, -0.59683486,
       -0.58863833, -0.61234011, -0.59846804, -0.51765996, -0.2775711 ,
       -0.56892186, -0.18237092, -0.23820257])

***
$\mathbf{\text{Cálculo da área sob os picos gaussianos}}$<br>
***
1.&emsp;A integral da gaussiana é
    $$ \int _{-\infty }^{\infty }a\,e^{-\left(x-b\right)^{2}/2c^{2}} \,dx = a \sqrt{2\pi}\left\vert c\right\vert $$
2.&emsp;A definição de FWHM é
    $$ \mathrm{FWHM}=2{\sqrt  {2\ln 2}}\;\sigma \approx 2.355\;\sigma. $$
3.&emsp;Portanto, o cálculo da área das contagens $A_C$ far-se-á por
    $$ A_C = a {\sqrt{2\pi} \over 2{\sqrt  {2\ln 2}}} \mathrm{FWHM} =
       {{1 \over 2}\sqrt{\pi \over {\ln 2}}}\cdot a \cdot \mathrm{FWHM} $$

In [70]:
sigmas = 1.0/(np.sqrt(2*(-a2)))
sigmas

array([4.00839792, 1.11852766, 1.82112035, 0.81475964, 0.9152883 ,
       0.92163877, 0.90362586, 0.91403857, 0.9827945 , 1.34214018,
       0.93747284, 1.65579744, 1.44881116])

In [71]:
fwhms = 2*np.sqrt(2*np.log(2))*sigmas
fwhms

array([9.43905577, 2.63393135, 4.2884107 , 1.91861234, 2.15533924,
       2.17029344, 2.12787628, 2.15239634, 2.31430418, 3.16049861,
       2.20757983, 3.89910501, 3.41168956])

In [72]:
chCentroids = -a1/(2*a2)
chCentroids

array([8.04985347e-01, 9.85542942e+01, 1.78493469e+02, 2.06699485e+02,
       3.12873776e+02, 3.25845471e+02, 3.63380933e+02, 3.71936420e+02,
       4.15047303e+02, 4.60063664e+02, 5.69232103e+02, 7.18471131e+02,
       8.13992377e+02])

In [73]:
alturas = np.exp(a0 - a1**2 / (4*a2))
alturas

array([1.22436699e+03, 5.70917508e+04, 3.24340149e+03, 9.36990720e+03,
       1.09813850e+03, 1.33308035e+05, 1.44153599e+03, 5.17161890e+03,
       1.35405588e+06, 7.26192761e+02, 4.36286269e+03, 5.79341476e+04,
       1.32216327e+04])

In [74]:
# Constante para cálculo da área do pico
karea = 0.5*np.sqrt(np.pi/np.log(2.0))
karea

1.0644670194312262

In [75]:
areas = karea * alturas * fwhms
areas

array([1.23019052e+04, 1.60070029e+05, 1.48057118e+04, 1.91361573e+04,
       2.51944547e+03, 3.07968994e+05, 3.26515704e+03, 1.18489801e+04,
       3.33571731e+06, 2.44309148e+03, 1.02522732e+04, 2.40453866e+05,
       4.80160914e+04])

In [76]:
nPks = len(polys_conv)

In [77]:
# O vértice da parábola tem coord xV = -b/2a, y = c - b^2/4a.
# O fwhm será 2 delta x, onde delta x é x(H/2)-x(vertice)
# Usando a função quadrática inversa, será raiz(b^2) etc

# Os canais centroides e as alturas serão
chCtroids = np.zeros(nPks)
heights = np.zeros(nPks)
chFwhms = np.zeros(nPks)
for iPk,coefs_pol in enumerate(polys_conv):
    a = coefs_pol[2]
    b = coefs_pol[1]
    c = coefs_pol[0]
    chCtroids[iPk] = -b / (2*a)
    heights[iPk] = np.exp( c - b**2 / (4*a))
    # Isso a seguir funciona mas é horrível...
    chFwhms[iPk] = - np.sqrt( b**2 - 4*a*( c - np.log(heights[iPk]/2) ) ) / a
enCtroids = spec2.pEn(chCtroids)
# print(enCtroids)
print(heights)
print(chFwhms)
print(chCtroids)
print(enCtroids)


[1.22436699e+03 5.70917508e+04 3.24340149e+03 9.36990720e+03
 1.09813850e+03 1.33308035e+05 1.44153599e+03 5.17161890e+03
 1.35405588e+06 7.26192761e+02 4.36286269e+03 5.79341476e+04
 1.32216327e+04]
[9.43905577 2.63393135 4.2884107  1.91861234 2.15533924 2.17029344
 2.12787628 2.15239634 2.31430418 3.16049861 2.20757983 3.89910501
 3.41168956]
[8.04985347e-01 9.85542942e+01 1.78493469e+02 2.06699485e+02
 3.12873776e+02 3.25845471e+02 3.63380933e+02 3.71936420e+02
 4.15047303e+02 4.60063664e+02 5.69232103e+02 7.18471131e+02
 8.13992377e+02]
[ -6.71394643  81.08075152 152.87906712 178.21263419 273.5743291
 285.2250106  318.93792952 326.62214173 365.34269091 405.77466843
 503.82559337 637.86639152 723.65992813]


In [78]:
# 2020-12-21 PAREI AQUI:
# Usar definições do markup language para recalcular FWHM (ver se bate) (sim, bate)
# E então achar as áreas dos picos
# Depois, montar arquivo com numpy.savetxt

In [79]:
fn2

'../Ipen/Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'

In [80]:
spec2.countDatetime

datetime.datetime(2018, 4, 10, 10, 58, 11)

In [81]:
str(spec2.countDatetime)

'2018-04-10 10:58:11'

In [82]:
spec2.spLVTime

5000.0

In [83]:
areas

array([1.23019052e+04, 1.60070029e+05, 1.48057118e+04, 1.91361573e+04,
       2.51944547e+03, 3.07968994e+05, 3.26515704e+03, 1.18489801e+04,
       3.33571731e+06, 2.44309148e+03, 1.02522732e+04, 2.40453866e+05,
       4.80160914e+04])

In [84]:
np.argsort(areas)[-5:]

array([12,  1, 11,  5,  8])

In [85]:
max5 = np.argsort(areas)[-5:]
maxareas = np.array(areas[max5])

In [86]:
np.resize(maxareas,7)

array([  48016.09143626,  160070.02850548,  240453.86586786,
        307968.99365977, 3335717.31103196,   48016.09143626,
        160070.02850548])

In [87]:
maxareas[0:2]=0
maxareas

array([      0.        ,       0.        ,  240453.86586786,
        307968.99365977, 3335717.31103196])

In [88]:
enCtroids[max5]

array([723.65992813,  81.08075152, 637.86639152, 285.2250106 ,
       365.34269091])

In [96]:
linha=np.concatenate((np.asarray([fn2,str(spec2.countDatetime),spec2.spLVTime]),areas[max5],enCtroids[max5]))
linha

array(['../Ipen/Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn',
       '2018-04-10 10:58:11', '5000.0', '48016.09143626338',
       '160070.02850548044', '240453.86586785677', '307968.99365977244',
       '3335717.3110319586', '723.6599281324906', '81.08075152452209',
       '637.8663915198699', '285.22501060259225', '365.3426909065381'],
      dtype='<U66')

In [100]:
numspecs = 1
all_polys_specs = []
for pol in all_polys_specs = []

    

NameError: name 'numspecs' is not defined

In [90]:
np.savetxt('test.out', ,newline=';')

SyntaxError: invalid syntax (<ipython-input-90-29801e2cdcb9>, line 1)

In [ ]:
np.concatenate(areas[max5],enCtroids[max5])

In [97]:
main_peaks_area_per_spec = np.empty([10,13], dtype=object)
main_peaks_area_per_spec

array([[None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None]], dtype=object)

In [99]:
main_peaks_area_per_spec[0]=linha
main_peaks_area_per_spec

array([['../Ipen/Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn',
        '2018-04-10 10:58:11', '5000.0', '48016.09143626338',
        '160070.02850548044', '240453.86586785677', '307968.99365977244',
        '3335717.3110319586', '723.6599281324906', '81.08075152452209',
        '637.8663915198699', '285.22501060259225', '365.3426909065381'],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None],
       [None, None, None, None, None, None, None, None, None, None, Non

In [ ]:
reconstru

In [ ]:
w45ywb45y w45y w4 

In [ ]:
xnxxnsd drhs drh

In [ ]:
# 2020-08-30 AQUI: Detecção dos picos
# 2020-08-30 Atenção ao parâmetro widths_range

In [ ]:
imag, pksCh, nscpk, chanCwtpk, iscaleCwtpk, maxcoefCwtpk, widths = spec2.pks_by_ridges(
    yNets, widths_range=(1,20), msnr=2.0, minCoefCwt=30.0)  
# Finalmente achei o parâmetro crítico: window_size. O default é muito grande!

In [ ]:
# 2020-08-31 ATENÇÃO AQUI: O pksCh obtido diretamente de find_pks_cwt não dá resultados bons;
# Picos importantes, próximos de picos maiores, não são detectados, mesmo com os kwargs
# bem ajustados, o que prejudica a delimitação das regiões.
# Vou usar os ridges criados com a análise de imagem (imag),para definir as regiões por meio dos retângulos
# e buscando o máximo cwt em cada ridge para selecionar os picos.

# As coordenadas dos pontos dos ridges estão em indices_tupl.
# Os canais dos picos estão em chanCwtpk.
# Os coeficientes cwt máximos estão em maxcoefCwtpk.


In [ ]:
indices_tupl = imag.nonzero()
np.asarray(indices_tupl).shape

In [ ]:
indices_tupl

In [ ]:
# 2020-08-30 Perfeito! É o que eu queria para listar indices dos elementos não nulos

In [ ]:
# 2020-09-06
# chanCwtpk define melhor os picos do que pksCh
len(chanCwtpk)

In [ ]:
chanCwtpk

In [ ]:
len(pksCh)

In [ ]:
sideChs = np.int_(np.round(widths[iscaleCwtpk]))
sideChs

In [ ]:
sideChs.shape

In [ ]:
# 2020-12-05
# O que segue desativei porque veio de um arquivo anterior.
# Verificar

In [ ]:
# 2020-09-11 Seleciona picos eliminando duplicados próximos, com base no coef cwt
# selectOverlappedPks está em opengamma_searchfuncs.py

In [ ]:
slctdPks=selectOverlappedPks(chanCwtpk, sideChs, maxcoefCwtpk)
slctdPks

In [ ]:
ma_chanCwtpk = chanCwtpk[slctdPks]
ma_maxcoefCwtpk = np.asarray(maxcoefCwtpk)[slctdPks]

In [ ]:
ma_chanCwtpk

In [ ]:
ma_chanCwtpk.size

In [ ]:
ma_maxcoefCwtpk

In [ ]:
ma_sideChs = sideChs[slctdPks]
ma_sideChs

In [ ]:
# 2020-09-12
# *** Rotina central para o ajuste dos picos do espectro ***
# Elimina picos com coef 2o grau positivo, antes da reconstrução.
# do espectro
# Descarta canais com menos que exp(2.5) ~ 12 contagens
# Fornece várias listas e a lista principal com os ajustes

# 2020-09-13
# O chanCwtpk *quase* sempre fornece a faixa de canais correta
# para fazer o ajuste.
# Mas às vezes dá problema no pico de aniilação, pois alarga demais 
# a faixa e o ajuste de 2o grau fica distorcido, por causa dos extremos da faixa.
# Ver isso.

In [ ]:
# 2020-12-06 Ver melhor o limiar 1.3 arbitrário em masklow
# 2020-12-15 Definir melhor win_for_plot
# 2020-12-15 Talvez usar numpy.trim_zeros em ypts antes do ajuste

hwin_for_plot = 15
reconstru = np.array(assembledBaseLine)
recons_multiline = [[],[]]
xs_multi_fits = []
ys_multi_fits = []
pksChans = []
pksParams = []
for i in range(ma_chanCwtpk.size):
    print('ma_chanCwtpk[i] = ', ma_chanCwtpk[i])
    sli = slice(ma_chanCwtpk[i]-hwin_for_plot, ma_chanCwtpk[i]+hwin_for_plot+1)
    # masklow = ma.masked_less(lnNetCounts[sli], 2.5).mask
    # masklow = ma.masked_less(lnNetCounts[sli], 1.3).mask
    masklow = ma.masked_less(lnNetCounts[sli], 0.0).mask
    xpts = np.array(range(sli.start, sli.stop))[~masklow]
    ypts = lnNetCounts[sli][~masklow]
    print('- ypts.shape[0] é ', ypts.shape[0])
    print('- xpts: ', xpts)
    print('- ypts: ', ypts)
    if ypts.shape[0] == 1:  # Que absurdo fazer isso. mask_less retorna array ou bool...
        xpts = xpts[0]
        ypts = ypts[0]
        print('- shape é 1')
    if ypts.size == hwin_for_plot:   # 2020-12-15 Horrível, mudar o qto antes
        p = P.fit(xpts, ypts, deg=2)
        print('- p.coef = ', p.coef)
        if p.coef[2] < 0:
            pksChans.append(ma_chanCwtpk[i])
            pksParams.append(p)
            ycalcs = np.exp(p(xpts))
            xs_multi_fits.append(xpts)
            ys_multi_fits.append(ycalcs)
            
            sli_for_plot = slice(ma_chanCwtpk[i]-hwin_for_plot, ma_chanCwtpk[i]+hwin_for_plot+1)
            print('-', sli_for_plot)
            if ((sli_for_plot.start >= 0) & (sli_for_plot.stop < nCh)):
                xs_for_plot = np.asarray(range(sli_for_plot.start, sli_for_plot.stop))
                ys_for_plot = np.exp(p(xs_for_plot))
                xs_for_plot_2 = np.asarray(np.linspace(xs_for_plot[0],xs_for_plot[-1],50))
                ys_for_plot_2 = np.exp(p(xs_for_plot_2))
                tck_step_for_plot = splrep(xs_for_plot, assembledBaseLine[sli_for_plot], k=3, s=0)
                step_for_plot = splev(xs_for_plot_2, tck_step_for_plot, der=0)
                reconstru[sli_for_plot] += ys_for_plot
                recons_multiline[0].append(xs_for_plot_2)
                recons_multiline[1].append(step_for_plot+ys_for_plot_2)

In [ ]:
reconstru.shape

In [ ]:
len(recons_multiline)

In [ ]:
ma_chanCwtpk.size

In [ ]:
len(pksParams)

In [ ]:
len(pksChans)

In [ ]:
# 2020-09-11 Bokeh graph

default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok5a = figure(
    plot_width =default_plot_width,
    plot_height=300,
    tools=tools
)
bok5a.title.text = 'Gráfico 5a: maxcoefCwtpk[i] nos ridges'
float_formatter = "{:n}".format
for i in range(len(chanCwtpk)):
    mytext = Label(x=chanCwtpk[i], y=iscaleCwtpk[i], text=float_formatter(chanCwtpk[i]),
                   text_font_size='10pt')
    bok5a.add_layout(mytext)
bok5a.circle(indices_tupl[1], indices_tupl[0], size=3)
bok5a.circle(chanCwtpk, iscaleCwtpk, size=8, line_color='magenta', fill_color='white', fill_alpha=0)

bok5b = figure(
    plot_width =default_plot_width,
    plot_height=300,
    y_axis_type="log",
    tools=tools
)
bok5b.title.text = '5b: picos p/ ridges nos canais chanCwtpk em círculos AZUIS no espectro líq yNets'
float_formatter = "{:.2f}".format
for i in range(ma_chanCwtpk.size):
    mytext = Label(x=ma_chanCwtpk[i], y=yNets[ma_chanCwtpk][i],
                   text=float_formatter(ma_maxcoefCwtpk[i]),
                   text_font_size='10pt')
    bok5b.add_layout(mytext)
bok5b.circle(ma_chanCwtpk, yNets[ma_chanCwtpk], size=8, line_color='blue', fill_color='white', fill_alpha=0)
    
bok5b.x_range = bok5a.x_range
bok5b.circle(spec2.xs, yNets, color='red',size=2)
bok5b.multi_line(xs_multi_fits, ys_multi_fits, color='blue')

# set up layout
p = gridplot([bok5a, bok5b], ncols=1)
show(p)

In [ ]:
# 2020-09-11 Bokeh graph

default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok6 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok6.title.text = 'Gráfico 6: tudo junto e misturado'
bok6.circle(spec2.xs, spec2.y0s, color="red",size=2.5)
bok6.line(spec2.xs, assembledBaseLine, color="black")
bok6.circle(spec2.xs, yNets, color='blue',size=1.5)
bok6.multi_line(xs_multi_fits, ys_multi_fits, color='blue')
bok6.line(spec2.xs, reconstru, color='green')

# set up layout
p = gridplot([bok6], ncols=1)
show(p)

In [ ]:
# 2020-12-06 Variante do gráfico 6

default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
bok6a = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok6a.title.text = 'Gráfico 6a: recons_multiline'
bok6a.circle(spec2.xs, spec2.y0s, color="red",size=2.5)
bok6a.line(spec2.xs, assembledBaseLine, color="black")
bok6a.circle(spec2.xs, yNets, color='blue',size=1.5)
bok6a.multi_line(recons_multiline[0], recons_multiline[1], color='orange')

# set up layout
p = gridplot([bok6a], ncols=1)
show(p)

In [ ]:
# 2020-08-15 AGORA interpretar os coefs. do ajuste gaussiano linearizado.
# 2020-09-12 Retomando

In [ ]:
nPks = len(xs_multi_fits)
nPks

In [ ]:
# Gr'afico do ajuste:

In [ ]:
default_plot_width, default_plot_height

In [ ]:
# PAREI AQUI a parte das paletas gr'aficas  2020-ago-12

In [ ]:
Paired6

In [ ]:
Pretos = ('#a6cee3', '#1f78b4', '#b2df8a', '#a6cee3', '#a6cee3', '#a6cee3')

In [ ]:
# 2020-12-14 Cálculo das áreas sob os picos: usa heights e chFwhms

***
$\mathbf{\text{Cálculo da área sob os picos gaussianos}}$<br>
***
1.&emsp;A integral da gaussiana é
    $$ \int _{-\infty }^{\infty }a\,e^{-\left(x-b\right)^{2}/2c^{2}} \,dx = a \sqrt{2\pi}\left\vert c\right\vert $$
2.&emsp;A definição de FWHM é
    $$ \mathrm{FWHM}=2{\sqrt  {2\ln 2}}\;\sigma \approx 2.355\;\sigma. $$
3.&emsp;Portanto, o cálculo da área das contagens $A_C$ far-se-á por
    $$ A_C = a {\sqrt{2\pi} \over 2{\sqrt  {2\ln 2}}} \mathrm{FWHM} =
       {{1 \over 2}\sqrt{\pi \over {\ln 2}}}\cdot a \cdot \mathrm{FWHM} $$

In [ ]:
# 2020-09-17
# Convertendo canais em energias
# ATENCAO NAO USAR MAIS POLYVAL, INVERTE OS COEFS

ens = spec2.coeffs_ChEn(spec2.xs)
# ens = np.polyval(spec2.coeffs_ChEn, spec2.xs)
enCtroids = spec2.coeffs_ChEn(chCtroids)
# enCtroids = np.polyval(spec2.coeffs_ChEn, chCtroids)
enCtroids

In [ ]:
# https://stackoverflow.com/questions/21008858/formatting-floats-in-a-numpy-array#21009774

# 2020-09-17 Bokeh graph
# Plot do espectro analisado em energias (keV)
default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset,save'
bok7 = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)
bok7.title.text = 'Gráfico 7: ' + fn2 \
    + '.   Início da medição: ' + str(spec2.countDatetime) \
    + '.   Live time: ' + str(spec2.spLVTime) \
    + '.   Real time: ' + str(spec2.spRLTime) + '.'
bok7.xaxis.axis_label = 'Energy (keV)'
bok7.xaxis.axis_label_text_font_size = '12pt'
bok7.xaxis.major_label_text_font_size = '10pt'

bok7.yaxis.axis_label = 'Counts'
bok7.yaxis.axis_label_text_font_size = '12pt'
bok7.yaxis.major_label_text_font_size = '10pt'

uai = ['{:.1f}'.format(i) for i in enCtroids]
source = ColumnDataSource(data=dict(height=heights,
                                   enkeV=enCtroids,
                                   lblPkEns=uai))
labels = LabelSet(x='enkeV', y='height', text='lblPkEns', level='glyph',
                  text_font_size = '10pt',
                  x_offset=5, y_offset=5, source=source, render_mode='canvas')

# boka.add_layout(mytext)
bok7.add_layout(labels)

bok7.circle(ens, spec2.y0s, color='orange',size=2.5)
bok7.line(ens, assembledBaseLine, color='brown')
bok7.line(ens, reconstru, color='turquoise')
# set up layout
p = gridplot([bok7], ncols=1)
show(p)

In [ ]:
erf(0.9)

In [ ]:
erf(0.0)

In [ ]:
teste = np.array([2,7,5,4,1,6])

In [ ]:
np.asarray(range(sli.start, sli.stop))

In [ ]:
men5 = ma.masked_less(teste,5)
men5

In [ ]:
teste[~men5.mask]

In [ ]:
teste[ma.masked_less(teste,5).mask]

In [ ]:
teste[~ma.masked_less(teste,5).mask]

In [ ]:
# 2020-09-13 Dicas legais para usar markdown no Jupyter-lab:
# medium.com/analytics-vidhya/writing-math-equations-in-jupyter-notebook-a-naive-introduction-a5ce87b9a214
# Ver a seguir:

# A primeira célula do par está como Raw para ver o código
# A segunda tem o mesmo conteúdo, mas está como Markdown

***
$\mathbf{\text{Esta é uma célula para testes do uso de markdown em Jupyter}}$<br>
***
1.&emsp;Initialize model with a constant value $$f_{0}(x) = \textrm{arg min}_{\gamma} \sum \limits _{i=1} ^{N} L(y_{i}, \gamma)$$
2.&emsp;For m = 1 to M:<br>
&emsp;&emsp;(a)&emsp;For $i = 1,2,...,N$ compute<br>
    $$r_{im} = - \displaystyle \Bigg[\frac{\partial L(y_{i}, f(x_{i}))}{\partial f(x_{i})}\Bigg]_{f=f_{m−1}}$$
&emsp;&emsp;(b)&emsp;Fit a regression tree to the targets $r_{im}$ giving terminal regions<br>
&emsp;&emsp;&emsp;&emsp;$R_{jm}, j = 1, 2, . . . , J_{m}.$<br><br>
&emsp;&emsp;(c)&emsp;For $j = 1, 2, . . . , J_{m}$ compute<br>
$$\gamma_{jm} = \underset{\gamma}{\textrm{arg min}} \sum \limits _{x_{i} \in R_{jm}} L(y_{i}, f_{m−1}(x_{i}) + \gamma)$$
<br>
&emsp;&emsp;(d)&emsp;Update $f_{m}(x) = f_{m−1}(x) + \sum _{j=1} ^{J_{m}} \gamma_{jm} I(x \in R_{jm})$<br><br>
3. Output $\hat{f}(x) = f_{M}(x)$
***

***
$\mathbf{\text{Gradient Tree Boosting Algorithm}}$<br>
***
1.&emsp;Initialize model with a constant value $$f_{0}(x) = \textrm{arg min}_{\gamma} \sum \limits _{i=1} ^{N} L(y_{i}, \gamma)$$
2.&emsp;For m = 1 to M:<br>
&emsp;&emsp;(a)&emsp;For $i = 1,2,...,N$ compute<br>
    $$r_{im} = - \displaystyle \Bigg[\frac{\partial L(y_{i}, f(x_{i}))}{\partial f(x_{i})}\Bigg]_{f=f_{m−1}}$$
&emsp;&emsp;(b)&emsp;Fit a regression tree to the targets $r_{im}$ giving terminal regions<br>
&emsp;&emsp;&emsp;&emsp;$R_{jm}, j = 1, 2, . . . , J_{m}.$<br><br>
&emsp;&emsp;(c)&emsp;For $j = 1, 2, . . . , J_{m}$ compute<br>
$$\gamma_{jm} = \underset{\gamma}{\textrm{arg min}} \sum \limits _{x_{i} \in R_{jm}} L(y_{i}, f_{m−1}(x_{i}) + \gamma)$$
<br>
&emsp;&emsp;(d)&emsp;Update $f_{m}(x) = f_{m−1}(x) + \sum _{j=1} ^{J_{m}} \gamma_{jm} I(x \in R_{jm})$<br><br>
3. Output $\hat{f}(x) = f_{M}(x)$
***

$\hat{Y} = \hat{\beta}_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j} $